In [1]:
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit # 使用 RandomizedSearchCV 和 TimeSeriesSplit
from sklearn.metrics import roc_auc_score, make_scorer, f1_score, recall_score, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier 
from sklearn.naive_bayes import GaussianNB
import numpy as np
import lightgbm as lgb

In [2]:
PROCESSED_DATA_DIR = Path("data/processing_checkpoint")
checkpoint_file_path = PROCESSED_DATA_DIR / "03_10_day_window_sliced.parquet"
df_final = pd.read_parquet(checkpoint_file_path)

In [3]:
def get_feature_types(df, target_col="label"):
    numerical_features = []
    categorical_features = []

    feature_columns = []
    for col in df.columns:
            if col == target_col or col == "userId" or pd.api.types.is_datetime64_any_dtype(df[col]):
                continue
            feature_columns.append(col)
        
    for col in feature_columns:
            if pd.api.types.is_numeric_dtype(df[col].dtype):
                numerical_features.append(col)
            else:
                categorical_features.append(col)

    return numerical_features, categorical_features

final_numerical_features, final_categorical_features = get_feature_types(df_final)

print(final_numerical_features, final_categorical_features )





['num_unique_artists', 'count_about', 'count_add_friend', 'count_add_to_playlist', 'count_cancel', 'count_cancellation_confirmation', 'count_downgrade', 'count_error', 'count_help', 'count_home', 'count_logout', 'count_nextsong', 'count_roll_advert', 'count_save_settings', 'count_settings', 'count_submit_downgrade', 'count_submit_upgrade', 'count_thumbs_down', 'count_thumbs_up', 'count_upgrade', 'count_total_sessions', 'user_lifecycle_h', 'ttl_length', 'item_per_session', 'frequency', 'avg_songs_session', 'thumbs_ratio', 'errors_per_session', 'ads_per_session', 'hours_since_last_session', 'active_days', 'active_days_ratio', 'session_length_variance', 'is_new_user', 'hours_since_downgrade', 'num_unique_songs', 'unique_songs_ratio', 'early_actions', 'late_actions', 'within_window_activity_ratio', 'within_window_activity_change', 'early_songs_played', 'late_songs_played', 'song_listening_change', 'recent_actions_last_3d', 'recent_activity_ratio', 'early_avg_items_per_session', 'late_avg_i

In [4]:
final_numerical_features = [
    'num_unique_artists', 'count_about', 'count_add_friend', 'count_add_to_playlist', 
    'count_downgrade', 'count_error', 
    'count_help', 'count_home', 'count_logout', 'count_nextsong', 'count_roll_advert', 
    'count_save_settings', 'count_settings', 'count_submit_downgrade', 
    'count_submit_upgrade', 'count_thumbs_down', 'count_thumbs_up', 'count_upgrade', 
    'count_total_sessions', 'user_lifecycle_h', 'ttl_length', 'item_per_session', 
    'frequency', 'avg_songs_session', 'thumbs_ratio', 'errors_per_session', 
    'ads_per_session', 'hours_since_last_session', 'active_days', 'active_days_ratio', 
    'session_length_variance', 'hours_since_downgrade', 
    'num_unique_songs', 'unique_songs_ratio', 'early_actions', 'late_actions', 
    'within_window_activity_ratio', 'within_window_activity_change', 
    'early_songs_played', 'late_songs_played', 'song_listening_change', 
    'recent_actions_last_3d', 'recent_activity_ratio', 'early_avg_items_per_session', 
    'late_avg_items_per_session', 'session_depth_change'
]

final_categorical_features = [
    'gender', 'last_level','is_new_user'
]



In [5]:
df_final["num_unique_songs"] = df_final["num_unique_songs"].fillna(0)

In [6]:
all_features = final_numerical_features + final_categorical_features
X = df_final[all_features]
y = df_final["label"]
print("Shape of Dataset loaded:", X.shape)

Shape of Dataset loaded: (60434, 49)


In [7]:
from sklearn.feature_selection import SelectFromModel

# Build LASSO selector for meaningful feature selection
lasso_selector = Pipeline([
    ("scaler", StandardScaler()),
    ("lasso", SelectFromModel(
        LogisticRegression(
            penalty="l1", solver="liblinear", C=0.1,
            class_weight="balanced", random_state=42
        )
    ))
])

lasso_selector.fit(X[final_numerical_features], y)

# Get Boolean mask from LASSO
selected_mask = lasso_selector.named_steps["lasso"].get_support()

# print out selected feature from LASSO
selected_num_features = []
for feature_name, keep_bool in zip(final_numerical_features, selected_mask):
    if keep_bool == True:
        selected_num_features.append(feature_name)

print("Selected num features:", selected_num_features)
print("Count of selected num features", len(selected_num_features))



Selected num features: ['num_unique_artists', 'count_about', 'count_add_friend', 'count_add_to_playlist', 'count_downgrade', 'count_error', 'count_help', 'count_home', 'count_logout', 'count_roll_advert', 'count_save_settings', 'count_settings', 'count_submit_downgrade', 'count_submit_upgrade', 'count_thumbs_down', 'count_thumbs_up', 'count_upgrade', 'count_total_sessions', 'user_lifecycle_h', 'item_per_session', 'frequency', 'avg_songs_session', 'thumbs_ratio', 'errors_per_session', 'ads_per_session', 'hours_since_last_session', 'active_days', 'active_days_ratio', 'session_length_variance', 'hours_since_downgrade', 'unique_songs_ratio', 'within_window_activity_ratio', 'within_window_activity_change', 'recent_actions_last_3d', 'recent_activity_ratio', 'late_avg_items_per_session', 'session_depth_change']
Count of selected num features 37


In [8]:
# Build final preprocessor with selected features

preprocessor_final = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), selected_num_features),
        ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), final_categorical_features)
    ],
    remainder="drop"
)

In [9]:
# Define Models and Random Search Space

# Base Pipeline
base_pipeline = ImbPipeline([
    ("preprocessor", preprocessor_final),
    ("classifier", LogisticRegression()) 
])

# Logistic Regression model
log_reg = LogisticRegression(
    penalty="l2",
    class_weight="balanced",
    solver="liblinear",
    random_state=42
)

# Random Forest
rf = RandomForestClassifier(
    class_weight="balanced_subsample",
    random_state=42
)

# XGB imbalance weight
neg = (y == 0).sum()
pos = (y == 1).sum()
xgb_scale = neg / pos

xgb = XGBClassifier(
    eval_metric="logloss",
    tree_method="hist",
    random_state=42
)

lgbm = lgb.LGBMClassifier(
    objective="binary",
    boosting_type="gbdt",
    n_estimators=300,
    learning_rate=0.05,
    class_weight="balanced",
    random_state=42
)

In [10]:
model_candidates = [
    {
        "classifier": [log_reg],
        "classifier__C": np.logspace(-2, 2, 5)
    },
    {
        "classifier": [rf],
        "classifier__n_estimators": [100, 200],
        "classifier__max_depth": [5, 10, 15]
    },
    {
        "classifier": [xgb],
        "classifier__learning_rate": [0.05, 0.1],
        "classifier__n_estimators": [100, 200],
        "classifier__max_depth": [3, 5],
        "classifier__scale_pos_weight": [xgb_scale]
    },
    {
        "classifier": [lgbm],
        "classifier__num_leaves": [31, 63, 127],
        "classifier__max_depth": [-1, 5, 10],
        "classifier__min_data_in_leaf": [20, 50, 100],
        "classifier__subsample": [0.7, 0.9, 1.0],
        "classifier__colsample_bytree": [0.7, 0.9, 1.0],
    }
]

In [11]:
# Group-Fold CV + Randomized Search

from sklearn.model_selection import GroupKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, make_scorer

groups = df_final["snapshot_day"]
cv = GroupKFold(n_splits=4)

random_search = RandomizedSearchCV(
    estimator=base_pipeline,
    param_distributions=model_candidates,
    n_iter=20,
    scoring="roc_auc",
    cv=cv.split(X, y, groups),
    verbose=2,
    n_jobs=-1,
    random_state=42,
    error_score='raise'   #check error
)

random_search.fit(X, y)

best_model = random_search.best_estimator_
best_score = random_search.best_score_

print("Best AUC:", best_score)
print("Best classifier:", best_model["classifier"])
print("Best params:", random_search.best_params_)

Fitting 4 folds for each of 20 candidates, totalling 80 fits


[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min

/Users/benedikt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[CV] END classifier=LGBMClassifier(class_weight='balanced', learning_rate=0.05, n_estimators=300,
               objective='binary', random_state=42), classifier__colsample_bytree=1.0, classifier__max_depth=5, classifier__min_data_in_leaf=20, classif

/Users/benedikt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/benedikt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/Users/benedikt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/benedikt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[CV] END classifier=LGBMClassifier(class_weight='balanced', learning_rate=0.05, n_estimators=300,
               objective='binary', random_state=42), classifier__colsample_bytree=0.7, classifier__max_

/Users/benedikt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[CV] END classifier=RandomForestClassifier(class_weight='balanced_subsample', random_state=42), classifier__max_depth=5, classifier__n_estimators=200; total time=  31.2s
[CV] END classifier=RandomForestClassifier(class_weight='balanced_subsample', random_state=42), classifier__max_depth=5, classifier__n_estimators=200; total time=  27.9s
[CV] END classifier=RandomForestClassifier(class_weight='balanced_subsample', random_state=42), classifier__max_depth=5, classifier__n_estimators=200; total time=  29.6s
[CV] END classifier=RandomForestClassifier(class_weight='balanced_subsample', random_state=42), classifier__max_depth=5, classifier__n_estimators=200; total time=  28.6s
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Info] Number of positive: 3090, number of negative: 57344
[L

In [12]:
PROCESSED_DATA_DIR = Path("data/processing_checkpoint")
checkpoint_file_path = PROCESSED_DATA_DIR / "04_user_base_test.parquet"
df_test = pd.read_parquet(checkpoint_file_path)
X_test = df_test[all_features]
y_pred = best_model.predict(X_test)
submission_df = pd.DataFrame({
    "id": df_test["userId"],
    "target": y_pred
})
submission_df.to_csv("submission_Jessie.csv", index=False)

print("Submission saved as submission_Jessie.csv")

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
Submission saved as submission_Jessie.csv


/Users/benedikt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/benedikt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [13]:
import pandas as pd

feature_names = best_model['preprocessor'].get_feature_names_out()
print("Final features count:", len(feature_names))
print(pd.DataFrame(feature_names, columns=["feature"]))

Final features count: 40
                               feature
0              num__num_unique_artists
1                     num__count_about
2                num__count_add_friend
3           num__count_add_to_playlist
4                 num__count_downgrade
5                     num__count_error
6                      num__count_help
7                      num__count_home
8                    num__count_logout
9               num__count_roll_advert
10            num__count_save_settings
11                 num__count_settings
12         num__count_submit_downgrade
13           num__count_submit_upgrade
14              num__count_thumbs_down
15                num__count_thumbs_up
16                  num__count_upgrade
17           num__count_total_sessions
18               num__user_lifecycle_h
19               num__item_per_session
20                      num__frequency
21              num__avg_songs_session
22                   num__thumbs_ratio
23             num__errors_per_session


In [14]:
print(df_final.groupby("snapshot_day")["label"].value_counts())


snapshot_day  label
9             0        13355
              1          821
19            0        14635
              1          817
29            0        14741
              1          789
39            0        14613
              1          663
Name: count, dtype: int64


In [15]:
print("Total positives in full dataset:", y.sum())
print(df_final.groupby("snapshot_day")["label"].sum())

Total positives in full dataset: 3090
snapshot_day
9     821
19    817
29    789
39    663
Name: label, dtype: int64


In [16]:
def evaluate_ensemble(lr_model, lxgb_model, X, y, groups):

    cv = GroupKFold(n_splits=4)
    best_auc = -1
    best_weight = 0.5

    for w in np.linspace(0, 1, 11):   # weights from 0.0 to 1.0
        aucs = []

        for train_idx, valid_idx in cv.split(X, y, groups):
            X_valid = X.iloc[valid_idx]
            y_valid = y.iloc[valid_idx]

            # Predict probability
            p_lr = lr_model.predict_proba(X_valid)[:, 1]
            p_xgb = lxgb_model.predict_proba(X_valid)[:, 1]

            # Weighted ensemble
            p_ens = w * p_lr + (1 - w) * p_xgb

            aucs.append(roc_auc_score(y_valid, p_ens))

        mean_auc = np.mean(aucs)
        print(f"weight_lr = {w:.1f}, AUC = {mean_auc:.5f}")

        if mean_auc > best_auc:
            best_auc = mean_auc
            best_weight = w

    print("Best ensemble weight:", best_weight)
    print("Best ensemble AUC:", best_auc)
    
    return best_weight, best_auc


In [17]:
preprocessor_2 = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", drop='first'), final_categorical_features),
        ("num", StandardScaler(), selected_num_features)
    ],
    remainder="drop"
)

lr_model = LogisticRegression(solver="liblinear", penalty="l1", C=0.15)

lr_pipeline = ImbPipeline(steps=[
    ("preprocessor", preprocessor_2),
    ('smote', SMOTE(random_state=42)),
    ("classifier", lr_model)
])


In [18]:
# 1. Fit LR pipeline first
lr_pipeline.fit(X, y)

# 2. Fit the tuned model (best_model already comes from RandomizedSearchCV)
best_model.fit(X, y)

# 3. Now ensemble evaluation is valid
best_w, best_ens_auc = evaluate_ensemble(
    lr_pipeline,
    best_model,
    X,
    y,
    groups=df_final["snapshot_day"]
)

print(best_w, best_ens_auc)


[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Info] Number of positive: 3090, number of negative: 57344
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001352 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5906
[LightGBM] [Info] Number of data points in the train set: 60434, number of used features: 40
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

/Users/benedikt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/benedikt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100


/Users/benedikt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/benedikt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
weight_lr = 0.0, AUC = 0.86962
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100


...

/Users/benedikt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/benedikt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
weight_lr = 0.9, AUC = 0.75512
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100


/Users/benedikt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/benedikt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
weight_lr = 1.0, AUC = 0.73654
Best ensemble weight: 0.0
Best ensemble AUC: 0.8696236451055194
0.0 0.8696236451055194


/Users/benedikt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/benedikt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [19]:
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score

groups = df_final["snapshot_day"]
cv = GroupKFold(n_splits=4)

lr_auc_scores = []

for train_idx, val_idx in cv.split(X, y, groups):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    lr_pipeline.fit(X_train, y_train)
    y_pred_proba = lr_pipeline.predict_proba(X_val)[:, 1]

    auc = roc_auc_score(y_val, y_pred_proba)
    lr_auc_scores.append(auc)

print("LR fold AUCs:", lr_auc_scores)
print("LR mean AUC:", sum(lr_auc_scores)/len(lr_auc_scores))


LR fold AUCs: [0.7410730905902155, 0.7304423969801271, 0.7409767269561731, 0.7081244804233316]
LR mean AUC: 0.7301541737374618


In [20]:
PROCESSED_DATA_DIR = Path("data/processing_checkpoint")
checkpoint_file_path = PROCESSED_DATA_DIR / "04_user_base_test.parquet"
df_test = pd.read_parquet(checkpoint_file_path)
X_test = df_test[all_features]
X_test = X_test.fillna({
    'gender': 'M',
    'num_unique_songs': 0
})

y_pred = lr_pipeline.predict(X_test)
submission_df = pd.DataFrame({
    "id": df_test["userId"],
    "target": y_pred
})
submission_df.to_csv("submission_lr_Jessie.csv", index=False)

print("Submission saved as submission_Jessie.csv")

Submission saved as submission_Jessie.csv


In [21]:
# Extract classifier
lgbm_clf = best_model.named_steps["classifier"]

# Extract feature names
preprocessor = best_model.named_steps["preprocessor"]
feature_names = preprocessor.get_feature_names_out()

# LightGBM importances
importance_gain = lgbm_clf.booster_.feature_importance(importance_type='gain')
importance_split = lgbm_clf.booster_.feature_importance(importance_type='split')

importance_df = pd.DataFrame({
    "feature": feature_names,
    "gain_importance": importance_gain,
    "split_importance": importance_split
}).sort_values("gain_importance", ascending=False)

display(importance_df.head(15))


,feature,gain_importance,split_importance
27,num__active_days_ratio,60298.214337,284
22,num__thumbs_ratio,17467.997234,360
24,num__ads_per_session,15813.764542,409
9,num__count_roll_advert,14850.299748,234
14,num__count_thumbs_down,10910.633997,162
25,num__hours_since_last_session,10758.245053,421
21,num__avg_songs_session,8944.825762,300
15,num__count_thumbs_up,8334.407408,262
29,num__hours_since_downgrade,8084.686315,174
19,num__item_per_session,7477.850551,281


In [24]:
lr_clf = lr_pipeline['classifier']

feature_names_lr = lr_pipeline['preprocessor'].get_feature_names_out()

coefficients = lr_clf.coef_[0]

lr_importance_df = pd.DataFrame({
    'feature': feature_names_lr,
    'coefficient': coefficients,
    'abs_coefficient': np.abs(coefficients)
}).sort_values('abs_coefficient', ascending=False)

print(lr_importance_df.head(15).to_markdown(index=False))

| feature                    |   coefficient |   abs_coefficient |
|:---------------------------|--------------:|------------------:|
| cat__last_level_paid       |      0.847302 |          0.847302 |
| num__count_thumbs_up       |     -0.81614  |          0.81614  |
| num__active_days_ratio     |      0.531582 |          0.531582 |
| num__ads_per_session       |      0.35399  |          0.35399  |
| num__active_days           |     -0.328019 |          0.328019 |
| num__count_add_to_playlist |      0.309814 |          0.309814 |
| num__num_unique_artists    |      0.277677 |          0.277677 |
| num__count_roll_advert     |      0.277317 |          0.277317 |
| num__count_downgrade       |      0.269587 |          0.269587 |
| num__count_upgrade         |     -0.261324 |          0.261324 |
| num__count_total_sessions  |      0.260688 |          0.260688 |
| num__hours_since_downgrade |     -0.23263  |          0.23263  |
| num__item_per_session      |     -0.215378 |          0.2153